In [27]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [3]:
train_data_path = "../../data/garments_train.csv"
test_data_path = "../../data/garments_test.csv"

In [4]:
train_df = pd.read_csv(train_data_path).loc[:, ["RawText", "Aspect", "SentimentPolarity"]].drop_duplicates().reset_index(drop=True)
train_df.head(2)

,RawText,Aspect,SentimentPolarity
0,이번에구매한데님은사이즈가잘맞네요 색상구성도괜찮고맘에든답니다 잘입겠습니다,사이즈,1
1,바지는 너무 편하고 좋은데 좀크게나온듯 그리고 허리고리 하나가 안달려서 밑단수선하면...,사이즈,-1


In [5]:
label_encoder = LabelEncoder()
enc_data = label_encoder.fit_transform(train_df["SentimentPolarity"])
num_labels = len(set(enc_data))

In [6]:
label_items = label_encoder.classes_
label_numbers = label_encoder.transform(label_items)
dict(zip(label_items, label_numbers))

{-1: 0, 0: 1, 1: 2}

In [7]:
X_train_text = train_df.loc[:, "RawText"].to_list()
X_train_pair = train_df.loc[:, "Aspect"].to_list()
y_train = enc_data

In [8]:
HUGGING_FACE_PATH = "klue/bert-base"
model = TFAutoModelForSequenceClassification.from_pretrained(HUGGING_FACE_PATH, num_labels=num_labels, from_pt=True)
tokenizer = AutoTokenizer.from_pretrained(HUGGING_FACE_PATH)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
X_train_encoding = tokenizer(
    text=X_train_text,
    text_pair=X_train_pair,
    padding=True,
    truncation=True,
    max_length=42,
    return_token_type_ids=True
)

In [11]:
SHUFFLE_PARAM = 1000

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train_encoding),
    y_train
)).shuffle(SHUFFLE_PARAM)

In [13]:
BATCH_PARAM = 32

validation_length = int(len(train_df) // 10)
train_except_val = train_dataset.skip(validation_length).batch(BATCH_PARAM)
validation_data = train_dataset.take(validation_length).batch(BATCH_PARAM)

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, metrics=["accuracy"])
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  110617344 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
Total params: 110619651 (421.98 MB)
Trainable params: 110619651 (421.98 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
model.fit(
    train_except_val,
    epochs=1,
    batch_size=BATCH_PARAM,
    validation_data=validation_data)

1267/1267 [==============================] - 472s 328ms/step - loss: 0.3650 - accuracy: 0.8623 - val_loss: 0.3089 - val_accuracy: 0.8732


In [16]:
test_df = pd.read_csv(test_data_path).loc[:, ["RawText", "Aspect", "SentimentPolarity"]].drop_duplicates().reset_index(drop=True)
test_df.head(2)

,RawText,Aspect,SentimentPolarity
0,바늘질 마감처리 불량. 싸구려 느낌이 팍팍. 털빠짐이 없다해서 구매했는데 털빠짐이 ...,품질,-1
1,바늘질 마감처리 불량. 싸구려 느낌이 팍팍. 털빠짐이 없다해서 구매했는데 털빠짐이 ...,기능,-1


In [17]:
X_test_text = test_df.loc[:, "RawText"].to_list()
X_test_pair = test_df.loc[:, "Aspect"].to_list()
y_test = label_encoder.transform(test_df.loc[:, "SentimentPolarity"].to_list())

In [18]:
X_test_encoding = tokenizer(
    text=X_test_text,
    text_pair=X_test_pair,
    padding=True,
    truncation=True,
    max_length=42,
    return_token_type_ids=True
)

In [21]:
test_dataset = tf.data.Dataset.from_tensor_slices(
    dict(X_test_encoding)
).batch(BATCH_PARAM)

In [22]:
predictions = model.predict(test_dataset)
predictions.logits

351/351 [==============================] - 42s 107ms/step


array([[ 2.924746 , -1.1598381, -1.4035499],
       [ 0.4341254, -0.7361781,  0.874065 ],
       [-2.6927867, -2.6060112,  4.555206 ],
       ...,
       [ 3.145392 , -1.023886 , -1.7061222],
       [ 2.8372338, -0.5972894, -1.5807097],
       [ 3.26579  , -1.1270336, -1.7857113]], dtype=float32)

In [25]:
y_pred = np.argmax(predictions.logits, axis=1)
y_pred

array([0, 2, 2, ..., 0, 0, 0])

In [28]:
accuracy_score(y_test, y_pred)

0.8628219983955789